In [ ]:
import random
import torch
import random
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
# !pip install tensorflow

In [ ]:
#load data from kaggle

import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
import kagglehub
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from transformers import get_scheduler

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# Load the dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv")

# Check the first few rows of the dataset
print(df.head())

   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  


In [ ]:
#check what's missing in the data

# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Check how many rows have missing data
rows_with_missing_values = df[df.isnull().any(axis=1)]

# Print out the missing values summary and rows with missing data
print("Missing values per column:\n", missing_values)
print("\nNumber of rows with missing values:", rows_with_missing_values.shape[0])

# Preview the rows with missing values
print("\nRows with missing values:")
print(rows_with_missing_values.head())

Missing values per column:
 Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

Number of rows with missing values: 597

Rows with missing values:
     Unnamed: 0 title                                               text  \
1             1   NaN     Did they post their votes for Hillary already?   
43           43   NaN  True. Hillary needs a distraction and what bet...   
162         162   NaN  All eyes on Electoral delegates. The People kn...   
185         185   NaN                                               Cool   
269         269   NaN  A leading US senator: US Supporting War in Syr...   

     label  
1        1  
43       1  
162      1  
185      1  
269      1  


In [ ]:
# Ensure that all values in 'text' are strings
processed_df = df.copy()
processed_df['text'] = df['text'].fillna('')  # Fill NaN values with an empty string
texts = processed_df['text'].astype(str).tolist()  # Convert to list of strings

In [ ]:
# Split the dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)
train_texts = train_texts[:500]
train_labels = train_labels[:500]
test_texts = test_texts[:500]
test_labels = test_labels[:500]

In [ ]:
# #clean the data of missing values

# # Ensure that all values in 'text' are strings
# df['text'] = df['text'].fillna('')  # Fill NaN values with an empty string
# texts = df['text'].astype(str).tolist()  # Convert to list of strings

# # map labels to binary values
# df['label'] = df['label'].map({'true': 1, 'false': 0})

# # Check the first few entries
# print(texts[:1])

# print(len(texts[0]))

# # Split the dataset
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['text'], df['label'], test_size=0.2, random_state=42
# )
# train_texts = train_texts[:500]
# train_labels = train_labels[:500]
# test_texts = test_texts[:500]
# test_labels = test_labels[:500]

['No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred t

In [ ]:
df['label']

,label
0,1
1,1
2,1
3,0
4,1
...,...
72129,0
72130,1
72131,0
72132,0


In [ ]:
#tokenize with bert
#mattias says to put this into batches (XXX - ?)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text, adjust the max_length parameter based on data analysis (XXX)
def encode_text(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Tokenize the text data
#train_encodings = encode_text(texts[:10])
train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(list(test_texts), padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(list(train_labels))
test_labels = torch.tensor(list(test_labels))

# Check the tokenized data
print(train_encodings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'input_ids': tensor([[  101,  2348, 26060,  ...,  2230,  1010,   102],
        [  101,  6221,  1046,  ...,  5050,  1996,   102],
        [  101,  3675,  6477,  ...,     0,     0,     0],
        ...,
        [  101, 22072,  3201,  ..., 26157,  7982,   102],
        [  101,  2047,  2259,  ..., 19506,  2036,   102],
        [  101,  2003,  2205,  ...,  1037, 14044,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [ ]:
# # Initialize the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Function to tokenize the text, adjust the max_length parameter based on data analysis (XXX)
# def encode_text(texts, max_length=512):  # Allow customization of max_length
#     return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='tf')

# # # Determine optimal max_length (e.g., using percentile of token lengths)
# # token_lengths = [len(tokenizer.encode(text)) for text in texts]
# # max_length = int(np.percentile(token_lengths, 95))  # Adjust percentile as needed

# # Tokenize the text data in batches
# batch_size = 32  # Adjust as needed
# all_encodings = []  # Store all encodings

# for i in range(0, len(texts), batch_size):
#     batch_texts = texts[i : i + batch_size]
#     batch_encodings = encode_text(batch_texts, max_length=512)  # Use determined max_length
#     all_encodings.append(batch_encodings)

# # Check a sample of the tokenized data
# print(all_encodings[0])  # Print the encodings for the first batch

In [ ]:

# Create a custom Dataset class
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]

# Prepare datasets and data loaders
train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Loss function
loss_fn = CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for batch in train_loader:
        inputs, labels = batch
        break

print(labels)

tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1])


In [ ]:
# labels = labels.to(device)
# labels.long()

# outputs.logits.long()

# loss_fn(outputs.logits, labels)

NameError: name 'outputs' is not defined

In [ ]:
# Training loop

for epoch in range(3):  # Train for 3 epochs
    print(f"Epoch {epoch + 1}")
    for batch in train_loader:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        print(f"Loss: {loss.item()}")

Epoch 1
Loss: 0.7237411737442017
Loss: 0.7529557943344116
Loss: 0.6579534411430359
Loss: 0.641806960105896
Loss: 0.6714503169059753
Loss: 0.6407596468925476
Loss: 0.5614355802536011
Loss: 0.5411689877510071
Loss: 0.5184589624404907
Loss: 0.47074976563453674
Loss: 0.4220966398715973
Loss: 0.23700210452079773
Loss: 0.3481229245662689
Loss: 0.2322738766670227
Loss: 0.2756775915622711
Loss: 0.5010231733322144
Loss: 0.29066649079322815
Loss: 0.21964502334594727
Loss: 0.3859632909297943
Loss: 0.3957614600658417
Loss: 0.10431446880102158
Loss: 0.16809171438217163
Loss: 0.19669045507907867
Loss: 0.2198384702205658
Loss: 0.2620452642440796
Loss: 0.196361243724823
Loss: 0.24514080584049225
Loss: 0.20585930347442627
Loss: 0.2178337126970291
Loss: 0.28868746757507324
Loss: 0.27670443058013916
Loss: 0.20817187428474426
Epoch 2
Loss: 0.1308790147304535
Loss: 0.16942256689071655
Loss: 0.023522939532995224
Loss: 0.030907373875379562
Loss: 0.114836685359478
Loss: 0.03168383613228798
Loss: 0.10213519632

In [ ]:
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Generate evaluation metrics
print(classification_report(true_labels, predictions, target_names=['False', 'True']))

In [ ]:
#save the model
model.save_pretrained('./fake_news_model')
tokenizer.save_pretrained('./fake_news_model')

In [ ]:
#tokenize with bert
#mattias says to put this into batches (XXX - ?)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text, adjust the max_length parameter based on data analysis (XXX)
def encode_text(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Tokenize the text data
#train_encodings = encode_text(texts[:10])
train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(list(test_texts), padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(list(train_labels))
test_labels = torch.tensor(list(test_labels))

# Check the tokenized data
print(train_encodings)